# Recreate Results

In [1]:
import os
import pickle
import re
import glob
import numpy as np
import pandas as pd 
import warnings
from copy import deepcopy
from causal_eval.ml_utils import COUNTVECTORIZER
from causal_eval.utils import *
from causal_eval.sampling import *

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score,
    average_precision_score
)

### Load Data

In [2]:
# Load Data
fname = './data/subpopA_physics_medicine.csv'
data = pd.read_csv(fname)

# Bag of words
vec = deepcopy(COUNTVECTORIZER)
all_texts = data['X'].to_numpy()
X = vec.fit_transform(all_texts).toarray()

# Set parameters
num_seeds_one_setting = 100
zeta0, zeta1, rct, random_state = 0.85, 0.15, 0.09557700480435129, 0
x, y, data_resampled_all_seeds = one_hyperparam_setting(data, zeta0, zeta1, num_seeds_one_setting)
data_resampled_dict_all_seeds = resampled_data_cleanup(data_resampled_all_seeds, vec)

### Explore paper | recreated results

In [3]:
# Choose to present either paper or recreated results
# path = "recreate"
path = "paper"

In [4]:
def explore(model, path):
    model_path = "results/" + path + "/" + model + ".pkl"
    # Load the data from the pickle file
    with open(model_path, "rb") as file:
        results = pickle.load(file)

    error = [x["relative_abs_error"] for x in results]
    print(f"Model : {model}")
    print(f"Relative Absolute Error  : {np.mean(error)} ({np.std(error)})")

    treatment = [x["treatment_test"] for x in results]
    outcome = [x["outcome_test"] for x in results]

    acc_treatment, acc_outcome = [], []
    for index in range(len(results)):

        # Load data subset treatment/outcome variables
        Y = data_resampled_dict_all_seeds[index]["Y"]
        T = data_resampled_dict_all_seeds[index]["T"]

        acc_treatment.append(accuracy_score(T, np.array((treatment[index] >= 0.5), dtype=int)))
        acc_outcome.append(accuracy_score(Y, np.array((outcome[index] >= 0.5), dtype=int)))

    print(f"Treatment Accuracy : {np.mean(acc_treatment)} | Outcome Accuracy : {np.mean(acc_outcome)}\n")

for model in ["adapter", "mpnet", "specter", "tf-idf"]:
    explore(model, path)

Model : adapter
Relative Absolute Error  : 0.10402119463778871 (0.08237854344945923)
Treatment Accuracy : 0.8318664730217751 | Outcome Accuracy : 0.9568820084921426

Model : mpnet
Relative Absolute Error  : 0.10311652298379201 (0.08110728944343315)
Treatment Accuracy : 0.8315963216960139 | Outcome Accuracy : 0.9567512172168311

Model : specter
Relative Absolute Error  : 0.10395782373410489 (0.08482731104385374)
Treatment Accuracy : 0.8277390786004232 | Outcome Accuracy : 0.9568534370576007

Model : tf-idf
Relative Absolute Error  : 0.11815261612606485 (0.09415519560174723)
Treatment Accuracy : 0.8063327509494593 | Outcome Accuracy : 0.9531067102252133

